In [3]:
import pandas as pd
import pandas.io.sql as sql
import jieba
import nltk
import jieba.posseg as pseg
from gensim import corpora, models, similarities
import re
import jieba.analyse as analyse
pd.set_option("display.max_rows", None)
from wordcloud import WordCloud
import matplotlib.pyplot as plt


## 小红书词云代码

In [7]:
dfred=pd.read_csv('red_top1000.csv')
#列出总共有多少个不同品牌（unique values of the brand column）
brands_total = dfred['brand'].unique()
print(brands_total)

['BEASTER' 'Dickies' 'HFP' '优时颜' '康宝 (CANBO)' '曼龙 (MALONE)'
 '糖果小姐 (Miss candy)' 'ADDICTION' 'XPEL' '斯图尔特韦茨曼 (Stuart Weitzman)'
 '桂格 (QUAKER)' '费列罗 (Ferrero Rocher)' '迪卡侬 (Decathlon)' '郁美净 (ZWILLING)'
 '锐度 (DUNK)' 'KIKO' 'NERDY' '创福康' '好欢螺 (HAOHUANLUO)' '御泥坊' '胡桃里' '至本'
 '蔓索 (Minenssey)' 'BOSE' 'Boots' 'REVOLUTION' 'Vida Glow' 'bagel'
 '三草两木 (SASELOMO)' '巴尔曼 (Balmain)' '海绵宝宝 (SpongeBob)' '统一'
 '罗杰·维维亚 Roger Vivier' '资生堂 (Shiseido)' '韩束 (KanS)' 'Kaws' '东芝 (TOSHIBA)'
 '沙宣 (Vidal Sassoon)' '爱肯拿 (Acana)' '玛速 (MASOOMAKE)' '珂润 (curel)'
 '花洛莉亚 (FLORTTE)' '长寿花 (LONGEVITY FLOWER)' 'THEV' 'VVC' 'oce' '徐福记'
 '桑姆·布郎尼 Thom Browne' '圣罗兰 (YSL)' '天猫精灵' '斐思妮 (FACELIVE)' '王饱饱' 'KIMTRUE'
 'Lululemon' 'OMTO' '卡帕 (KAPPA)' '巴利 (Bally)' '晓姿 (AXXZIA)'
 '林家铺子 (LEASUN FOOD)' '特仑苏' '范斯 (VANS)' '贝恩施 (beiens)' '宝丽 (POLA)' '旺旺'
 '李医生 (DOCTOR LI)' '松达 (SONGDA)' '空中飞人 (Air Jordan)' '红谷 (HONGU)' '香飘飘'
 '魅族 (meizu)' 'UKISS' '兰芝 (LANEIGE)' '力士 (LUX)' '卫龙' '名创优品 (MINISO)'
 '罗莱 (luolai)' '野小兽 (YESOUL)' 'HABA' 'Self-Portra

In [8]:
dfred.head()

brand                   note_id  \
0  BEASTER  608f64710000000021036b18   
1  BEASTER  60965fd90000000001025d41   
2  BEASTER  6055d8fb000000000102bff8   
3  BEASTER  6086867b000000002103ba0e   
4  BEASTER  600112270000000001000d91   

                                             content  likes  rnk  
0  OOTD丨夏季蓝色系穿搭丨破洞裤绝绝子！本人16390\n衬衫：Beaster（s）颜色还挺...   5524    1  
1  🎁520送男友礼物推荐～最近刷到很多姐妹都在为520送男朋友什么礼物发愁[吧唧R]送男朋友比...   4635    2  
2  OOTD｜今天好韩哦🍳咸菜绿情侣T恤好高级✨好喜欢现在可以乱穿衣服的季节！！\n这件咸菜绿的...   4417    3  
3  坐我腿上还是坐我腰上？-周末去逛了一下泡桐树街，里面开了非常多不错的咖啡小店，#PickCo...   3351    4  
4  不看必后悔系列✨103家明星同款国潮韩潮店[害羞R][害羞R]我以前是不穿潮牌的，觉得这种小...   3277    5

In [10]:
#删除content是空的那些行
dfred = dfred[dfred['content'].notna()]

###文本清洗（清洗后的column名字为clean_cmt）
import re
def process_text(text):
    text = re.sub(r"[0-9]+", '', text) 
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    text = ''.join(e for e in text if e.isalnum())
    return text

content = dfred['content'].dropna() #把空的content去掉
processed_content = []
for i in range(len(content)):
    processed_content.append([process_text(content.iloc[i])])


In [11]:
import numpy as np
dfred["clean_cmt"] = np.array(processed_content)

In [ ]:
###加入stopwords
ss = pd.read_excel('red词频统计_标注.xlsx',engine = 'openpyxl')
stopwords = ss[ss['Unnamed: 2'] == 1]
stopwordlist = list(stopwords['Unnamed: 0'])
stopwordlist.extend(['飞吻','笑哭','偷笑','哭惹','派对','微笑','害羞'])

#创建content dataframe，列名称：text，内容：文本处理后的content
content = pd.DataFrame(processed_content,columns = ['text'])

adj = []
allow_pos = ('na') #选出形容词&名词
for i in range(len(content)):
    lines = content.iloc[i].values.tolist()
    content1 = "".join(lines)
    words = jieba.analyse.extract_tags(content1, topK=50, withWeight=False, allowPOS=(allow_pos))
    for w in words:
        if w in stopwordlist:
            words.remove(w)
    adj.append([i,",".join(words)])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.748 seconds.
Prefix dict has been built successfully.


In [ ]:
#在原dataframe的基础上更新dataframe：加入一列包含所有形容词&名词
dfred['adj'] = adj

In [ ]:
dfred.head()

brand                   note_id  \
0  BEASTER  608f64710000000021036b18   
1  BEASTER  60965fd90000000001025d41   
2  BEASTER  6055d8fb000000000102bff8   
3  BEASTER  6086867b000000002103ba0e   
4  BEASTER  600112270000000001000d91   

                                             content  likes  rnk  \
0  OOTD丨夏季蓝色系穿搭丨破洞裤绝绝子！本人16390\n衬衫：Beaster（s）颜色还挺...   5524    1   
1  🎁520送男友礼物推荐～最近刷到很多姐妹都在为520送男朋友什么礼物发愁[吧唧R]送男朋友比...   4635    2   
2  OOTD｜今天好韩哦🍳咸菜绿情侣T恤好高级✨好喜欢现在可以乱穿衣服的季节！！\n这件咸菜绿的...   4417    3   
3  坐我腿上还是坐我腰上？-周末去逛了一下泡桐树街，里面开了非常多不错的咖啡小店，#PickCo...   3351    4   
4  不看必后悔系列✨103家明星同款国潮韩潮店[害羞R][害羞R]我以前是不穿潮牌的，觉得这种小...   3277    5   

                                           clean_cmt  \
0  ootd丨夏季蓝色系穿搭丨破洞裤绝绝子本人衬衫beasters颜色还挺衬肤色的polo短袖不...   
1  送男友礼物推荐最近刷到很多姐妹都在为送男朋友什么礼物发愁吧唧r送男朋友比送姐妹礼物难多了每次...   
2  ootd今天好韩哦咸菜绿情侣t恤好高级好喜欢现在可以乱穿衣服的季节这件咸菜绿的长袖t好高级感...   
3  坐我腿上还是坐我腰上周末去逛了一下泡桐树街里面开了非常多不错的咖啡小店pickcoffee就...   
4  不看必后悔系列家明星同款国潮韩潮店害羞r害羞r我以前是不穿潮牌的觉得这种小众原创设计肯定特别...   

                                                 adj  
0   [0, 破洞,裤绝,绝子,底鞋,厚底,小个子,牛仔裤,衬衫,肤色,友好,蓝色,白色,颜色,基础]  
1  [1, 姐妹,衣服,狗子,壳子,男朋友,耳麦,剃须刀,礼物,耳机,时候,码数,情侣装,一束花...  
2  [2, 长袖,咸菜,恤好,洋气,牙子,长裙,情侣,穿衣服,男孩子,高高,裙子,店里,不规则,...  
3  [3, 恶魔,衬衫,版型,国潮,垂感,恤和,泡桐树,短靴,印花,鬼脸,项链,小店,门面,厚实...  
4  [4, 国潮,潮牌,小鬼,追星,同款,衣舍,造梦者,弹壳,情侣,粉丝,偶像,金子,功夫,女孩...

In [ ]:
###此处以第一个品牌示例
brand0 = []
for index,row in dfred.iterrows():
    if brands_total[0] in str(row['brands']):  #将所有是此品牌的row摘出
        brand0.append(row['adj']) #将品牌对应的切词结果摘出
        
text = ",".join(i for i in adj)
# 注意：改成自己的字体路径！！！！！
font = r'AaLingGanHei45J-2.ttf'   # 改成相应的字体的路径即可

wc1600by1000 = WordCloud(collocations=False, background_color='white',font_path=font, max_words=2000,width=1600, height=1000, margin=4).generate(text.lower())
wc800by500 = WordCloud(collocations=False, background_color='white',font_path=font, max_words=2000,width=800, height=500, margin=4).generate(text.lower())

plt.imshow(wc1600by1000)
plt.imshow(wc800by500)
plt.axis("off")
plt.show()
#wc.to_file('redcmt.png')  # 把词云保存下来

KeyError: 'brands'

In [ ]:
###（后期to-do：将小红书1000个品牌，每个品牌分开；每个品牌做一个词云function call）
#for index,row in dfred.iterrows():
    #if brands_total[i] in str(row['brands']): 
        #词云function call